In [1]:
!git clone https://github.com/VAST-AI-Research/TripoSR.git
!pip install --upgrade setuptools
!pip install -q onnxruntime

fatal: destination path 'TripoSR' already exists and is not an empty directory.


In [2]:
!pip install -q rembg

In [3]:
%cd /content/TripoSR
!pip install -r requirements.txt

/content/TripoSR
  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-rcdxb_ve
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-rcdxb_ve
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3381600ddc3d2e4d74222f8495866be5fafbace4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.34.0
    Uninstalling huggingface-hub-0.34.0:
      Successfully uninstalled huggingface-hub-0.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.16.0 requires huggingface_hub>=0

In [4]:
!pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.34.1 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.0 which is incompatible.


In [5]:
import requests, time, subprocess
from PIL import Image
from io import BytesIO
import gc
from google.colab import userdata
import cv2
import torch

In [6]:
HF_TOKEN = userdata.get('HF_TOKEN') #or paste below
#HF_TOKEN = "your_hf_token"
API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"

In [7]:
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
clear_memory()

In [27]:
from rembg import remove
def remove_background(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    output = remove(img_byte_arr)
    result_image = Image.open(BytesIO(output))
    white_bg = Image.new('RGB', result_image.size, (255, 255, 255))
    if result_image.mode == 'RGBA':
        white_bg.paste(result_image, mask=result_image.split()[3])
    else:
        white_bg.paste(result_image)
    return white_bg

def generate_image_from_prompt(prompt):
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    enhanced_prompt = f"{prompt}, highly detailed, professional product photography, studio lighting, white background, single object, clean, sharp focus, 8k resolution, photorealistic, centered composition, no shadows, isolated object"

    payload = {
        "inputs": enhanced_prompt,
        "parameters": {
            "height": 512,
            "width": 512,
            "num_inference_steps": 28,
            "guidance_scale": 3.5
        }
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image = remove_background(image)
        image_path = "generated_image.png"
        image.save(image_path)
        return image_path
    else:
        raise Exception(f"Image generation failed: {response.status_code}, {response.text}")

def generate_3d_mesh(input_source, is_text=True, output_dir="output"):
    if is_text:
        image_path = generate_image_from_prompt(input_source)
    else:
        image_path = input_source

    process = subprocess.run(
        ["python3", "run.py", image_path, "--output-dir", output_dir],
        capture_output=True,
        text=True
    )

    # Print stdout and stderr
    if process.stdout:
        print("STDOUT:", process.stdout)
    if process.stderr:
        print("STDERR:", process.stderr)

    print(f"Return code: {process.returncode}")
    print(f"3D mesh generated at: {output_dir}")

In [ ]:
generate_3d_mesh("A king chair", is_text=True)      # From text pro

In [35]:
generate_3d_mesh("/content/TripoSR/examples/unicorn.png", is_text=False)

STDERR: 2025-07-25 16:32:33,110 - INFO - Initializing model ...
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2025-07-25 16:32:42,305 - INFO - Initializing model finished in 9195.24ms.
2025-07-25 16:32:42,305 - INFO - Processing images ...
2025-07-25 16:32:43,077 - INFO - Processing images fi

In [15]:
#!python3 run.py "/content/Tree (2).png" --output-dir output/